# **HuggingFace and LangChain RAG to query your own PDF Files**

Sample code to POC using HuggingFace and LangChain to read your own PDF file and ask questions.

**1. Install necessary libraries**

These libraries are essential for:

**transformers and sentence-transformers:** To work with Hugging Face models for text embedding.

**faiss-gpu:** For efficient similarity search within the document embeddings. If running this locally on your laptop change to faiss-cpu

**langchain:** For building the RAG pipeline.
pypdf: For loading and processing the PDF document.


In [1]:
!pip install transformers sentence-transformers faiss-gpu langchain pypdf langchain_community


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 255.2/255.2 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.5/294.5 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.4/404.4 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/

**2. Load the PDF document**

This code utilizes PyPDFLoader from LangChain to load the content of your PDF file into documents.

In [11]:
from langchain.document_loaders import PyPDFLoader

# Following lines of code to read the PDF from your Google drive account. Comment if reading from elsewhere.
from google.colab import drive
drive.mount('/content/drive')

loader = PyPDFLoader("/content/drive/My Drive/your_pdf_file.pdf")  # Replace with your PDF file path
documents = loader.load()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**3. Split the document into chunks**

This step breaks down the PDF content into smaller chunks using RecursiveCharacterTextSplitter to ensure manageable processing. chunk_size and chunk_overlap can be adjusted.

In [12]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
texts = text_splitter.split_documents(documents)

**4. Create embeddings for the chunks**

This code initializes a Hugging Face embedding model using HuggingFaceEmbeddings and specifies the model to use (all-mpnet-base-v2 in this case) for generating text embeddings.

In [13]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


**5. Store embeddings in a vector store**

This uses FAISS to create a vector store, enabling efficient storage and retrieval of document chunks based on their embeddings

In [14]:
from langchain.vectorstores import FAISS

db = FAISS.from_documents(texts, embeddings)

**6. Create a retrieval QA chain**

This sets up the RetrievalQA chain, utilizing a specified language model (google/flan-t5-xl here) for answering questions and the db vector store for retrieving relevant information. Make sure to set your Hugging Face API token.

In [15]:
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFaceHub

from google.colab import userdata

sec_key = userdata.get('HUGGINGFACEHUB_API_TOKEN')

import os
os.environ['HUGGINGFACEHUB_API_TOKEN'] = sec_key

llm = HuggingFaceHub(repo_id="google/flan-t5-large", model_kwargs={"temperature":0.5, "max_length":512})
# Remember to set your Hugging Face API token as an environment variable named HUGGINGFACEHUB_API_TOKEN
# or pass it as token argument.
# flan-t5-large is a decent model to experiment with. Runs fine locally on CPU as well.
# xl and xxl versions of the model take a lot of resources and will consume your daily free limit on Colab.

qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=db.as_retriever())

**7. Query the RAG system**

This code demonstrates how to query the RAG system with a question. Replace "What is the content of the document about?" with your desired query to get relevant answers based on the PDF content.

This comprehensive approach enables you to build a robust RAG application for querying PDF documents. Remember to replace placeholders like "your_pdf_file.pdf" with your actual file path and customize parameters as needed.

In [16]:
query = "what is the content of the document about?"
result = qa.run(query)
print(result)

(ii)
